In [ ]:
# # # Main
# %pip install -q Levenshtein
# %pip install -q trl accelerate
# %pip install -q huggingface_hub
# %pip install -q bert-score
# %pip install -q sentence-transformers
# %pip install -q unsloth

In [ ]:
!huggingface-cli login --token <token_hf>

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: read).
The token `Colab` has been saved to /teamspace/studios/this_studio/.cache/huggingface/stored_tokens
Your token has been saved to /teamspace/studios/this_studio/.cache/huggingface/token
Login successful.
The current active token is: `Colab`


In [2]:
import unsloth
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from unsloth import is_bfloat16_supported
import torch
import numpy as np
import re
import time
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from Levenshtein import ratio
from collections import defaultdict
from trl import ORPOConfig, ORPOTrainer
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

print(unsloth.__version__)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
2025.8.1


In [3]:
class CONFIG:
    seed = 42
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
    # reward_model_name = "nicolinho/QRM-Llama3.1-8B-v2"
    benchmark_name = "openai/gsm8k"
    # dataset_name = "mlabonne/orpo-dpo-mix-40k"

    train_data_size = 1600
    benchmark_subset_size = 50
    max_seq_length = 2048
    train_batch_size = 2
    gradient_accumulation_steps = 4
    epochs = 1

    # LoRA Configs
    lora_rank = 64 ,
    lora_alpha = 64 ,
    use_gradient_checkpointing = "unsloth"
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    # dpo_output_dir = "llama-3.2-3b-dpo-checkpoint"
    # orpo_output_dir = "llama-3.2-3b-orpo-checkpoint"

device = CONFIG.device

In [4]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

def load_model_and_tokenizer(model_id, max_seq_length, inference=False):
    print("Loading model and tokenizer using unsloth...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_id,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )

    # Apply Llama-specific chat template
    if not hasattr(tokenizer, 'chat_template'):
        print("Applying chat template for Llama model...")
        # Get the chat template for Llama 3.1
        tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
        
    
    # Use left padding for generation
    tokenizer.padding_side = "left"
    
    # Add a proper pad token instead of using eos
    if tokenizer.pad_token is None:
        # Use a special pad token or add one
        if "<|pad|>" not in tokenizer.get_vocab():
            tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
            model.resize_token_embeddings(len(tokenizer))
        else:
            tokenizer.pad_token = "<|pad|>"
    
    # Set pad_token_id consistently
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
    model.config.pad_token_id = tokenizer.pad_token_id
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    
    if inference:
        FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

    return model, tokenizer

In [5]:
CONFIG.model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

main_model, main_tokenizer = load_model_and_tokenizer(CONFIG.model_name, CONFIG.max_seq_length,inference=True)
# rewriter_model, rewriter_tokenizer = main_model, main_tokenizer
rewriter_model, rewriter_tokenizer = load_model_and_tokenizer(CONFIG.model_name, CONFIG.max_seq_length)

Loading model and tokenizer using unsloth...
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 21.964 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading model and tokenizer using unsloth...
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 21.964 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Load Benchmark

In [6]:
def load_gsm8k_dataset(benchmark_name, config_name="main"):
    """Load the GSM8K dataset from HuggingFace."""
    dataset = load_dataset(benchmark_name, config_name, split="test")
    return dataset

def create_train_set(dataset, num_trains=100 , num_tests=50 , seed=CONFIG.seed):
    """Create a fixed sample dataset for evaluation."""

    # Shuffle the dataset with the given seed
    shuffled_dataset = dataset.shuffle(seed=seed)

    # Select examples for few-shot learning
    test_dataset = shuffled_dataset.select(range(num_tests))

    # Select test samples for evaluation
    train_dataset = shuffled_dataset.select(range(num_tests, num_tests + num_trains))

    print(f"Created {len(train_dataset)} train samples and {len(test_dataset)} test samples")

    return train_dataset, test_dataset

In [ ]:
# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load dataset
dataset = load_gsm8k_dataset(CONFIG.benchmark_name)

# Select subset
train_set , test_set = create_train_set(dataset,num_trains=50)

# Display one sample
print(f"Question: {train_set[0]['question']}")
print(f"\nAnswer: {train_set[0]['answer']}")

In [7]:
# # save datasets

# import os

# os.makedirs("./dataset", exist_ok=True)
# train_set.save_to_disk("./dataset/train_set")
# test_set.save_to_disk("./dataset/test_set")

In [7]:
# load datasets
from datasets import load_from_disk

train_set = load_from_disk("./dataset/train_set")
test_set = load_from_disk("./dataset/test_set")

In [8]:
# for i in range(2):
#     # Display one sample
#     print(f"Question: {train_set[i]['question']}")
#     print(f"\nAnswer: {train_set[i]['answer']}")

### Prompt Engineering

In [9]:
import re

def extract_gt_answer(text):
    # The regex pattern looks for a line that starts with optional whitespace,
    # followed by '####', optional whitespace, and then captures one or more digits.
    pattern = r"^\s*####\s*(\d+)"
    # Use the MULTILINE flag so the pattern matches each line.
    match = re.search(pattern, text, re.MULTILINE)
    if match:
        return match.group(1)  # Return the captured group, e.g. "109"
    else:
        # Fallback approach: if the "####" line isn't present, try to grab the last computed result
        # by finding all matches of the pattern <<expression=result>>.
        matches = re.findall(r"<<[^>]+=(\d+)>>", text)
        if matches:
            return matches[-1]  # Return the last result found
    return None

def extract_gen_answer(text):
    """
    Extracts the first answer between <Answer> and </Answer> tags in the input text.
    Returns the answer as a string, or None if not found.
    """
    match = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None 


In [ ]:
from typing import Any, Optional
import torch

def create_prompts(
    question: str,
    model: Any,
    tokenizer: Any = None,
    *,
    style: str = "concise",
    domain_hint: Optional[str] = None,
    temperature: float = 0.1,
    max_tokens: int = 1024,
) -> str:
    """
    rewrite `question` so a *different* model can answer it as accurately as possible.
    
    Args:
        question: the original, possibly messy user question
        model: loaded model instance (unsloth, transformers pipeline, etc.)
        tokenizer: tokenizer if needed for the model
        style: rewrite style; choose between 'concise', 'detailed', 'bullet', etc.
        domain_hint: optional domain (e.g. 'medical', 'finance') to bias the rewrite
        temperature: generation temperature
        max_tokens: max tokens to generate
    
    Returns:
        rewritten question as a plain string
    """
    system_msg = (
        "You are an expert prompt engineer. "
        "Rewrite the user's question so that *another* LLM will answer it "
        "with maximum accuracy and minimal ambiguity."
    )
    if domain_hint:
        system_msg += f" The topic domain is {domain_hint}."
    system_msg += f" Rewrite style: {style}. Output only the rewritten question."
    
    # create chat messages format for llama-3.1 template
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"User question: {question}\n\nRewritten question:"}
    ]
    
    # try different model interfaces
    try:
        # check if it's an unsloth model with tokenizer
        if tokenizer is not None and hasattr(model, 'generate'):
            # apply chat template to messages
            formatted_prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )
            
            # tokenize
            inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
            if torch.cuda.is_available():
                inputs = inputs.cuda()
            # print(f"Inputs shape: {inputs}")
            # generate
            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_new_tokens=max_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )
            
            # decode only the new tokens
            response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
            return response.strip()
            
        # check if it's a transformers pipeline
        elif hasattr(model, '__call__') and hasattr(model, 'task'):
            # for pipelines, use the formatted messages directly
            response = model(
                messages,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                return_full_text=False
            )
            if isinstance(response, list) and len(response) > 0:
                return response[0]['generated_text'].strip()
            return str(response).strip()
            
        # check if it has a generate method (general transformers model)
        elif hasattr(model, 'generate') and tokenizer is not None:
            formatted_prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )
            inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
            if torch.cuda.is_available():
                inputs = inputs.cuda()
            
            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_new_tokens=max_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=tokenizer.pad_token_id
                )
            response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
            return response.strip()
            
        # fallback - try calling it directly
        else:
            response = model(messages)
            return str(response).strip()
            
    except Exception as e:
        print(f"Error calling model: {e}")
        # return original question if rewriting fails
        return question

# def gsm8k_prompt(question):
#     final_prompt = f'''Write the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>

# Question: {question}'''

#     return final_prompt

def gsm8k_prompt(question,tokenizer=None):
    message = [{"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context.\n\
Write the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n"},
    {"role": "user", "content": f"Question: {question}"}
    ]
    if tokenizer is not None:
        return tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
    
    message = f'''Write the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>

Question: {question}'''

    return message

In [11]:
# print(gsm8k_prompt("How many times does 7 go into 49?", rewriter_tokenizer))

In [12]:
# from unsloth import FastLanguageModel
# from unsloth.chat_templates import get_chat_template

# # use it to rewrite questions
# raw_question = train_set[0]['question']
# better_question = create_prompts(
#     raw_question, 
#     rewriter_model, 
#     rewriter_tokenizer,
#     style="direct",
#     domain_hint="mathematics",
# )

# print("original:", raw_question)
# print("rewritten:", better_question)

In [13]:
# print(gsm8k_prompt(better_question))

In [14]:
# main_model.eval()
# prompt = gsm8k_prompt(raw_question)
# inputs = main_tokenizer(prompt, return_tensors="pt")
            
# input_ids = inputs.input_ids.to(device)
# attention_mask = inputs.attention_mask.to(device)

# # Generate text
# with torch.no_grad():
#     outputs = main_model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         max_new_tokens=1024,
#         # temperature=0.0,
#         do_sample=False,
#         use_cache=True,
#         pad_token_id=main_tokenizer.pad_token_id,
#         eos_token_id=main_tokenizer.eos_token_id
#         # num_return_sequences=1
#     )
# # Decode the generated text
# response = main_tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
# print(response)

In [15]:
# print(extract_gen_answer(response))

In [16]:
# extract_gt_answer(train_set[0]['answer'])

In [17]:
# print(gsm8k_prompt(better_question))

In [18]:
# main_model.eval()
# prompt = gsm8k_prompt(better_question)
# inputs = main_tokenizer(prompt, return_tensors="pt")

# input_ids = inputs.input_ids.to(device)
# attention_mask = inputs.attention_mask.to(device)

# # Generate text
# with torch.no_grad():
#     outputs = main_model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         max_new_tokens=1024,
#         # temperature=0.0,
#         do_sample=False,
#         use_cache=True,
#         pad_token_id=main_tokenizer.pad_token_id,
#         eos_token_id=main_tokenizer.eos_token_id
#         # num_return_sequences=1
#     )
# # Decode the generated text
# response = main_tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
# print(response)

In [19]:
# print(extract_gen_answer(response))

### Evaluate Prompts

In [20]:
import random
from tqdm import tqdm

def evaluate_prompts(
    model,
    tokenizer,
    sample_dataset,
    model_revision=None,
    tokenizer_revision=None,
    rewrite_prompts=False,
    seed=42,
    max_gen_tokens=1024,
    show_progress=True,
    return_details=False,
):
    """
    Evaluates the model on a set of math problems with or without prompt rewriting.
    
    Args:
        model, tokenizer: loaded model and tokenizer.
        train_set: list-like with 'question', 'answer' fields.
        rewrite_prompts: if True, use create_prompts to rewrite each question.
        seed: for reproducibility.
        max_gen_tokens: max generation length.
        show_progress: whether to show tqdm progress bar.
        return_details: if True, return a detailed result list.
    
    Returns:
        accuracy: float, ratio of correct answers.
        details: (optional) list of dicts with info per sample.
    """
    np.random.seed(seed)
    random.seed(seed)
    results = []
    num_tokens = 0

    iterator = tqdm(sample_dataset, desc="Evaluating", disable=not show_progress)
    for sample in iterator:
        orig_question = sample['question']
        gt_answer = extract_gt_answer(sample['answer'])
        
        # Prompt rewriting if enabled
        if rewrite_prompts:
            prompt = create_prompts(
                orig_question, model_revision, tokenizer_revision,
                style="direct", domain_hint="math word problem"
            )

        else:
            prompt = orig_question
        
        num_tokens += len(tokenizer.encode(prompt))
        # Wrap question in final prompt for model
        full_prompt = gsm8k_prompt(prompt, tokenizer=tokenizer)
        
        # Format for model input (Llama-3 style)
        # chat_msg = [
        #     {"role": "user", "content": full_prompt}
        # ]
        # formatted = tokenizer.apply_chat_template(
        #     chat_msg, tokenize=False, add_generation_prompt=True
        # )
        inputs = tokenizer(full_prompt, return_tensors="pt").to(CONFIG.device)
        
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=max_gen_tokens,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False,
            )
        gen_text = tokenizer.decode(output_ids[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        pred_answer = extract_gen_answer(gen_text)
        
        correct = (str(pred_answer).strip() == str(gt_answer).strip())
        results.append({
            "question": orig_question,
            "rewritten": prompt if rewrite_prompts else None,
            "full_prompt": full_prompt,
            "gt_answer": gt_answer,
            "gen_text": gen_text,
            "pred_answer": pred_answer,
            "correct": correct,
        })
        
    accuracy = np.mean([r["correct"] for r in results])
    if return_details:
        return accuracy, results , num_tokens
    else:
        return accuracy


In [21]:
# test_set = load_gsm8k_dataset(CONFIG.benchmark_name)
# test_set = test_set.shuffle(seed=42)
# test_set = test_set.select(range(100, 150))

# train_set

In [23]:
acc_vanilla, details_vanilla, total_tokens_vanilla = evaluate_prompts(main_model, main_tokenizer, test_set, rewrite_prompts=False, return_details=True)
print(f"Accuracy without rewriting: {acc_vanilla:.3f}")

Evaluating: 100%|██████████| 50/50 [05:20<00:00,  6.42s/it]

Accuracy without rewriting: 0.380


In [24]:
print(f"Total tokens without rewriting: {total_tokens_vanilla}")

Total tokens without rewriting: 3055


In [25]:
acc_rewrite, details_rewrite, total_tokens_rewrite = evaluate_prompts(main_model, main_tokenizer, test_set, model_revision=rewriter_model, tokenizer_revision=rewriter_tokenizer, rewrite_prompts=True, return_details=True)
print(f"Accuracy with rewriting: {acc_rewrite:.3f}")

Evaluating: 100%|██████████| 50/50 [06:24<00:00,  7.68s/it]

Accuracy with rewriting: 0.420


In [48]:
print(f"Total tokens with rewriting: {total_tokens_rewrite}")

Total tokens with rewriting: 2141


In [27]:
# def extract_xml_answer(text: str) -> str:
#     answer = text.split("<answer>")[-1]
#     answer = answer.split("</answer>")[0]
#     return answer.strip()

In [22]:
# # Install TRL if you haven’t already
# %pip install trl accelerate

from datasets import Dataset, load_dataset
from trl import GRPOConfig, GRPOTrainer
import torch
import pandas as pd

In [23]:
from bert_score import score
from sentence_transformers import SentenceTransformer, util
import torch

def compute_reward(
    orig_prompt,
    rewritten_prompt,
    main_model,
    main_tokenizer,
    gt_answer,
    target_len=60,
    w_acc=2.0, w_sem=0.3, w_len=0.1
):
    # Generate answer from rewritten prompt
    qa_prompt = gsm8k_prompt(rewritten_prompt, tokenizer=main_tokenizer)
    inputs = main_tokenizer(qa_prompt, return_tensors="pt").to(main_model.device)
    with torch.no_grad():
        outputs = main_model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=512,
            pad_token_id=main_tokenizer.pad_token_id,
            eos_token_id=main_tokenizer.eos_token_id,
            do_sample=False
        )
    gen_text = main_tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True
    )
    answer = extract_gen_answer(gen_text)
    is_correct = (str(answer).strip() == str(gt_answer).strip())
    acc_reward = 1.0 if is_correct else -1.0

    # Compute semantic similarity via BERTScore F1
    P, R, F1 = score(
        [orig_prompt],
        [rewritten_prompt],
        lang='en',
        rescale_with_baseline=True
    )
    sem_sim = F1[0].item()

    # Length penalty
    rew_len = len(main_tokenizer.tokenize(rewritten_prompt))
    len_penalty = abs(rew_len - target_len) / target_len

    # Aggregate reward
    reward = w_acc * acc_reward + w_sem * sem_sim - w_len * len_penalty
    return reward, {
        "acc_reward": acc_reward,
        "sem_sim": sem_sim,
        "len_penalty": len_penalty,
        "answer": answer
    }

In [25]:
train_set

Dataset({
    features: ['question', 'answer'],
    num_rows: 50
})

In [26]:
def system_prompt(
    question: str,
    style: str = "direct",
    domain_hint: Optional[str] = None,
) -> str:
    """
    Create a system prompt for rewriting a question.
    
    Args:
        question: the original user question
        model: loaded model instance (unsloth, transformers pipeline, etc.)
        tokenizer: tokenizer if needed for the model
        style: rewrite style; choose between 'concise', 'detailed', 'bullet', etc.
        domain_hint: optional domain (e.g. 'medical', 'finance') to bias the rewrite
    
    Returns:
        rewritten question as a plain string
    """
    # create system message for prompt engineering
    system_msg = (
        "You are an expert prompt engineer.\n"
        "Rewrite the user's question so that *another* LLM will answer it "
        "with maximum accuracy and minimal ambiguity.\n"
        "Also you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar."
    )
    # if domain_hint:
    #     system_msg += f" The topic domain is {domain_hint}."
    # system_msg += f" Rewrite style: {style}. Output only the rewritten question."
    
    system_msg += f" Output only the rewritten question."
    
    # create chat messages format for llama-3.1 template
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"User question: {question}\n\nRewritten question:"}
    ] 
    return messages

print(system_prompt(question="How many times does 7 go into 49?",
    style="direct",
    domain_hint="math word problem"
))

[{'role': 'system', 'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question."}, {'role': 'user', 'content': 'User question: How many times does 7 go into 49?\n\nRewritten question:'}]


In [27]:
# SYSTEM_PROMPT = "You are a helpful assistant that answers questions based on the provided context.\n\
# Write the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n"

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(datast) -> Dataset:
    # data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = datast.map(lambda x: { # type: ignore
        'prompt': 
            system_prompt(question=x['question'],
                style="direct",
                domain_hint="math word problem"
            ),
        'answer': extract_gt_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions(train_set)

In [28]:
train_set

Dataset({
    features: ['question', 'answer'],
    num_rows: 50
})

In [29]:
dataset[0]

{'question': 'Jerry is twice as old as he was 5 years ago. How old will Jerry be in 3 years?',
 'answer': '13',
 'prompt': [{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question.",
   'role': 'system'},
  {'content': 'User question: Jerry is twice as old as he was 5 years ago. How old will Jerry be in 3 years?\n\nRewritten question:',
   'role': 'user'}]}

In [30]:
# # Assuming train_set is already loaded
# train_set = train_set.rename_column("question", "prompt")

In [31]:
def grpo_reward_fn(completions, prompts, answer, **kwargs):
    """
    GRPOTrainer will pass:
      - completions: list of generated strings (one per prompt in the batch)
      - prompts:       list of original prompt strings
    We compute a scalar reward per completion.
    """
    rewards = []
    for prompt, completion in zip(prompts, completions):
        # compute_reward returns (reward_scalar, metrics_dict)
        print(f"prompt:  {prompt}  --------- completion:  {completion}   ------------ answer:  {answer[0]}")
        reward, details = compute_reward(
            orig_prompt=prompt[-1]["content"].replace("User question:","").replace("\n\nRewritten question:","").strip(),
            rewritten_prompt=completion[0]["content"],
            main_model=main_model,
            main_tokenizer=main_tokenizer,
            gt_answer=answer[0],  # adjust if needed
        )
        print('-'*20, f"Question:\n{prompt}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{completion}", f"\nExtracted:\n{details['answer']}")
        rewards.append(float(reward))
    return rewards

In [32]:
rewriter_model = FastLanguageModel.get_peft_model(
    rewriter_model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

Unsloth 2025.8.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [33]:
max_prompt_length = 256
max_seq_length = CONFIG.max_seq_length

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 6,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    # max_steps = 50,
    save_steps = 10,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

In [34]:
trainer = GRPOTrainer(
    model = rewriter_model,
    processing_class = rewriter_tokenizer,
    reward_funcs = [
        grpo_reward_fn,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()
trainer.save_model("llama_rewriter_grpo")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 131072, 'temperature': 0.6, 'top_p': 0.9}. If this is not desired, please set these values explicitly.


prompt:  [{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question.", 'role': 'system'}, {'content': 'User question: The expenditure of Joseph in May was $500. In June, his expenditure was $60 less. How much was his total expenditure for those two months?\n\nRewritten question:', 'role': 'user'}]  --------- completion:  [{'role': 'assistant', 'content': "What was Joseph's total expenditure in May and June combined, given that his May expenditure was $500 and his June expenditure was $60 less than his May expenditure?"}]   ------------ answer:  940


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


-------------------- Question:
[{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question.", 'role': 'system'}, {'content': 'User question: The expenditure of Joseph in May was $500. In June, his expenditure was $60 less. How much was his total expenditure for those two months?\n\nRewritten question:', 'role': 'user'}] 
Answer:
940 
Response:
[{'role': 'assistant', 'content': "What was Joseph's total expenditure in May and June combined, given that his May expenditure was $500 and his June expenditure was $60 less than his May expenditure?"}] 
Extracted:
940
prompt:  [{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximu

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

-------------------- Question:
[{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question.", 'role': 'system'}, {'content': 'User question: Jerry is twice as old as he was 5 years ago. How old will Jerry be in 3 years?\n\nRewritten question:', 'role': 'user'}] 
Answer:
13 
Response:
[{'role': 'assistant', 'content': "What will be Jerry's age in three years, given that his current age is twice his age five years ago?"}] 
Extracted:
13
prompt:  [{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


-------------------- Question:
[{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question.", 'role': 'system'}, {'content': 'User question: Jerry is twice as old as he was 5 years ago. How old will Jerry be in 3 years?\n\nRewritten question:', 'role': 'user'}] 
Answer:
13 
Response:
[{'role': 'assistant', 'content': "What will be Jerry's age in 3 years, given that his current age is twice his age 5 years ago?"}] 
Extracted:
13
prompt:  [{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


-------------------- Question:
[{'content': "You are an expert prompt engineer.\nRewrite the user's question so that *another* LLM will answer it with maximum accuracy and minimal ambiguity.\nAlso you can use prompt engineering methods like incontext learning, few-shot prompting, and chain-of-thought reasoning with proper punctuation marks and grammar. Output only the rewritten question.", 'role': 'system'}, {'content': 'User question: Jerry is twice as old as he was 5 years ago. How old will Jerry be in 3 years?\n\nRewritten question:', 'role': 'user'}] 
Answer:
13 
Response:
[{'role': 'assistant', 'content': "What will be Jerry's age in 3 years, given that his current age is twice his age 5 years ago?"}] 
Extracted:
13


In [38]:
acc_rewrite_grpo, details_rewrite_grpo, total_tokens_grpo = evaluate_prompts(main_model, main_tokenizer, test_set, model_revision=rewriter_model, tokenizer_revision=rewriter_tokenizer, rewrite_prompts=True, return_details=True)
print(f"Accuracy with rewriting: {acc_rewrite_grpo:.3f}")

Evaluating: 100%|██████████| 50/50 [07:02<00:00,  8.46s/it]

Accuracy with rewriting: 0.440


In [39]:
print(f"Total tokens with rewriting (GRPO): {total_tokens_grpo}")

Total tokens with rewriting (GRPO): 2168


In [40]:
details_rewrite_grpo

[{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.",
  'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?",
  'full_prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 05 Aug 2025\n\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer><|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
  'gt_answer': '109',
  'gen_text': "To solve this problem, we need to first find the current ages of 

In [55]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(details_rewrite_grpo[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?", 'full_prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 05 Aug 2025\n\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer><|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n", 'gt_answer': '109', 'gen_text': "To solve this problem, we need to first find the current ages o

### Test other models

In [ ]:
# unsloth/Qwen2.5-3B-Instruct-bnb-4bit
# unsloth/gemma-3-4b-it-unsloth-bnb-4bit

#### Qwen2.5-3B-Instruct-bnb-4bit

In [25]:
model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"

qwen_model, qwen_tokenizer = load_model_and_tokenizer(model_name, CONFIG.max_seq_length,inference=True)

Loading model and tokenizer using unsloth...
==((====))==  Unsloth 2025.8.1: Fast Qwen2 patching. Transformers: 4.54.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 21.964 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


##### Without rewriting

In [26]:
qwen_acc_vanilla, qwen_details_vanilla, qwen_total_tokens_vanilla = evaluate_prompts(qwen_model, qwen_tokenizer, test_set, rewrite_prompts=False, return_details=True)
print(f"Accuracy without rewriting (Qwen): {qwen_acc_vanilla:.3f}")

Evaluating: 100%|██████████| 50/50 [10:08<00:00, 12.16s/it]

Accuracy without rewriting (Qwen): 0.500


In [27]:
print(f"Total tokens without rewriting (Qwen): {qwen_total_tokens_vanilla}")

Total tokens without rewriting (Qwen): 3120


In [28]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(qwen_details_vanilla[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': None, 'full_prompt': "<|im_start|>system\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n<|im_end|>\n<|im_start|>user\nQuestion: Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.<|im_end|>\n<|im_start|>assistant\n", 'gt_answer': '109', 'gen_text': "To find Allen's age 10 years from now, we first need to determine the current ages of Darrell and Allen.\n\nGiven:\n- The ratio of Darrell's age to Allen's age is 7:11.\n- Their total age is 162.\n\nLet's denote Darrell's age as \\( D \\) and Allen's age as \\( A \\).\n\nFrom the given ratio, we can write:\n\\[ \\frac{D}{A} = \\frac{7}{11} \\]\n\nThis implies:\n\\[ D = 7k \

##### Before GRPO training with rewrite

In [29]:
qwen_acc_rewrite, qwen_details_rewrite, qwen_total_tokens_rewrite = evaluate_prompts(qwen_model, qwen_tokenizer, test_set, model_revision=rewriter_model, tokenizer_revision=rewriter_tokenizer, rewrite_prompts=True, return_details=True)
print(f"Accuracy with rewriting (Qwen): {qwen_acc_rewrite:.3f}")

Evaluating: 100%|██████████| 50/50 [10:51<00:00, 13.04s/it]

Accuracy with rewriting (Qwen): 0.360


In [30]:
print(f"Total tokens with rewriting (Qwen): {qwen_total_tokens_rewrite}")

Total tokens with rewriting (Qwen): 2214


In [31]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(qwen_details_rewrite[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total age is 162?", 'full_prompt': "<|im_start|>system\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n<|im_end|>\n<|im_start|>user\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total age is 162?<|im_end|>\n<|im_start|>assistant\n", 'gt_answer': '109', 'gen_text': "To find Allen's age 10 years from now, we first need to determine their current ages. We know that the sum of their ages is 162, and their ages are in the ratio 7:11.\n\nLet's denote Darrell's current age as \\( D \\) and Allen's cu

##### After GRPO training with rewrite

In [42]:
qwen_acc_rewrite_grpo, qwen_details_rewrite_grpo, qwen_total_tokens_grpo = evaluate_prompts(qwen_model, qwen_tokenizer, test_set, model_revision=rewriter_model, tokenizer_revision=rewriter_tokenizer, rewrite_prompts=True, return_details=True)
print(f"Accuracy with rewriting: {qwen_acc_rewrite_grpo:.3f}")

Evaluating: 100%|██████████| 50/50 [11:36<00:00, 13.92s/it]

Accuracy with rewriting: 0.320


In [43]:
print(f"Total tokens with rewriting (Qwen-GRPO): {qwen_total_tokens_grpo}")

Total tokens with rewriting (Qwen-GRPO): 2245


In [44]:
qwen_details_rewrite_grpo

[{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.",
  'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?",
  'full_prompt': "<|im_start|>system\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n<|im_end|>\n<|im_start|>user\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?<|im_end|>\n<|im_start|>assistant\n",
  'gt_answer': '109',
  'gen_text': "To find Allen's age 10 years from now, we first need to determine their current ages. We know that the sum of their current ages is 162, and their ages are in the ratio 7:11.\n\nLet's denote Darrell's current age as \\

In [53]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(qwen_details_rewrite_grpo[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?", 'full_prompt': "<|im_start|>system\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n<|im_end|>\n<|im_start|>user\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?<|im_end|>\n<|im_start|>assistant\n", 'gt_answer': '109', 'gen_text': "To find Allen's age 10 years from now, we first need to determine their current ages. We know that the sum of their current ages is 162, and their ages are in the ratio 7:11.\n\nLet's denote Darrell's current age as 

#### Gemma-3-4b-it-unsloth-bnb-4bit

In [32]:
model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit"

# 1. Load model + processor (formerly 'tokenizer')
gemma_model, gemma_processor = FastLanguageModel.from_pretrained(
    model_name= model_name,
    max_seq_length=CONFIG.max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

# 2. Grab the real Hugging Face tokenizer out of the processor
gemma_tokenizer = gemma_processor.tokenizer  # now a PreTrainedTokenizerFast :contentReference[oaicite:2]{index=2}

# 3. Apply any chat template you need
if not hasattr(gemma_tokenizer, "chat_template"):
    print("Applying chat template for Llama model...")
    gemma_tokenizer = get_chat_template(gemma_tokenizer, chat_template="llama-3.1")  # or "gemma-3" :contentReference[oaicite:3]{index=3}

# 4. Left‐padding for generation
gemma_tokenizer.padding_side = "left"

# 5. Ensure a proper pad token
if gemma_tokenizer.pad_token is None:
    gemma_tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    gemma_model.resize_token_embeddings(len(gemma_tokenizer))

# 6. Set pad_token_id on both tokenizer and model configs
pad_id = gemma_tokenizer.convert_tokens_to_ids(gemma_tokenizer.pad_token)
gemma_tokenizer.pad_token_id = pad_id
gemma_model.config.pad_token_id = pad_id
gemma_model.generation_config.pad_token_id = pad_id

# 7. (Optional) if other code expects processor.tokenizer, put it back
gemma_processor.tokenizer = gemma_model

# 8. Enable fast inference
FastLanguageModel.for_inference(gemma_model)

==((====))==  Unsloth 2025.8.1: Fast Gemma3 patching. Transformers: 4.54.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 21.964 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwi

##### Without rewriting

In [33]:
gemma_acc_vanilla, gemma_details_vanilla, gemma_total_tokens_vanilla = evaluate_prompts(gemma_model, gemma_tokenizer, test_set, rewrite_prompts=False, return_details=True)
print(f"Accuracy without rewriting (Gemma): {gemma_acc_vanilla:.3f}")

Evaluating: 100%|██████████| 50/50 [35:14<00:00, 42.28s/it] 

Accuracy without rewriting (Gemma): 0.700


In [34]:
print(f"Total tokens without rewriting (Gemma): {gemma_total_tokens_vanilla}")

Total tokens without rewriting (Gemma): 3141


In [35]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(gemma_details_vanilla[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': None, 'full_prompt': "<bos><start_of_turn>user\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n\n\nQuestion: Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.<end_of_turn>\n<start_of_turn>model\n", 'gt_answer': '109', 'gen_text': "Let Darrell's age be 7x and Allen's age be 11x.\nThe total age is 7x + 11x = 18x.\nWe are given that the total age is 162, so 18x = 162.\nDividing both sides by 18, we get x = 162 / 18 = 9.\nSo, Darrell's age is 7 * 9 = 63 and Allen's age is 11 * 9 = 99.\nAllen's age now is 99.\nIn 10 years, Allen's age will be 99 + 10 = 109.\n\n<answer>109</answer>", 'pred_answer': '109', 'correct': True}
Q

##### Before GRPO training with rewrite

In [36]:
gemma_acc_rewrite, gemma_details_rewrite, gemma_total_tokens_rewrite = evaluate_prompts(gemma_model, gemma_tokenizer, test_set, model_revision=rewriter_model, tokenizer_revision=rewriter_tokenizer, rewrite_prompts=True, return_details=True)
print(f"Accuracy with rewriting (Gemma): {gemma_acc_rewrite:.3f}")

Evaluating: 100%|██████████| 50/50 [31:55<00:00, 38.31s/it]  

Accuracy with rewriting (Gemma): 0.560


In [37]:
print(f"Total tokens with rewriting (Gemma): {gemma_total_tokens_rewrite}")

Total tokens with rewriting (Gemma): 2273


In [38]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(gemma_details_rewrite[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total age is 162?", 'full_prompt': "<bos><start_of_turn>user\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n\n\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total age is 162?<end_of_turn>\n<start_of_turn>model\n", 'gt_answer': '109', 'gen_text': "Let Allen's current age be $A$ and Darrell's current age be $D$.\nWe are given that the ratio of their ages is $7:11$, so $\\frac{A}{D} = \\frac{7}{11}$.\nWe are also given that their total age is 162, so $A + D = 162$.\n\nFrom the ratio, we have $11

##### After GRPO training with rewrite

In [48]:
gemma_acc_rewrite_grpo, gemma_details_rewrite_grpo, gemma_total_tokens_grpo = evaluate_prompts(gemma_model, gemma_tokenizer, test_set, model_revision=rewriter_model, tokenizer_revision=rewriter_tokenizer, rewrite_prompts=True, return_details=True)
print(f"Accuracy with rewriting: {gemma_acc_rewrite_grpo:.3f}")

Evaluating: 100%|██████████| 50/50 [32:03<00:00, 38.46s/it]  

Accuracy with rewriting: 0.600


In [56]:
print(f"Total tokens with rewriting (Gemma-GRPO): {gemma_total_tokens_grpo}")

Total tokens with rewriting (Gemma-GRPO): 2304


In [50]:
gemma_details_rewrite_grpo

[{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.",
  'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?",
  'full_prompt': "<bos><start_of_turn>user\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n\n\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?<end_of_turn>\n<start_of_turn>model\n",
  'gt_answer': '109',
  'gen_text': "Let Allen's current age be $A$ and Darrell's current age be $D$.\nWe are given that the ratio of their current ages is $7:11$, so $\\frac{A}{D} = \\frac{7}{11}$.\nWe are also given that their total current age is 162, so $A + D = 162$.\

In [54]:
n = 5

for i in range(n):
    print(f"Question {i+1}")
    print(gemma_details_rewrite_grpo[i])

Question 1
{'question': "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.", 'rewritten': "What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?", 'full_prompt': "<bos><start_of_turn>user\nYou are a helpful assistant that answers questions based on the provided context.\nWrite the final numeric answer between <answer> and </answer> tags, like this: <answer>123</answer>\n\n\n\nQuestion: What is Allen's age 10 years from now, given that Darrell's and Allen's current ages are in the ratio 7:11 and their total current age is 162?<end_of_turn>\n<start_of_turn>model\n", 'gt_answer': '109', 'gen_text': "Let Allen's current age be $A$ and Darrell's current age be $D$.\nWe are given that the ratio of their current ages is $7:11$, so $\\frac{A}{D} = \\frac{7}{11}$.\nWe are also given that their total current age is 162, so $A + D = 162$

### Others

In [42]:
# import copy

# # make an exact, independent copy
# ref_model = copy.deepcopy(rewriter_model)

# rewriter_model = FastLanguageModel.get_peft_model(
#     rewriter_model,
#     r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 64,
#     lora_dropout = 0, # Currently only supports dropout = 0
#     bias = "none",    # Currently only supports bias = "none"
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

# # One must patch the DPO Trainer first!
# from unsloth import PatchDPOTrainer
# PatchDPOTrainer()

In [43]:
# %pip install -q sentence-transformers

In [44]:
# # # Load the base (math‑answer) model
# # main_model, main_tokenizer = FastLanguageModel.from_pretrained(
# #     model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
# #     max_seq_length=2048,
# #     dtype=None,
# #     load_in_4bit=True
# # )
# # main_tokenizer = get_chat_template(main_tokenizer, chat_template="llama-3.1")
# # # Pad token handling
# # main_tokenizer.padding_side = "left"
# # if main_tokenizer.pad_token is None:
# #     if "<|pad|>" not in main_tokenizer.get_vocab():
# #         main_tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
# #         main_model.resize_token_embeddings(len(main_tokenizer))
# #     else:
# #         main_tokenizer.pad_token = "<|pad|>"
# # main_tokenizer.pad_token_id = main_tokenizer.convert_tokens_to_ids(main_tokenizer.pad_token)
# # main_model.config.pad_token_id = main_tokenizer.pad_token_id
# # main_model.generation_config.pad_token_id = main_tokenizer.pad_token_id

# # # Rewriter model (clone of base model); this will be fine‑tuned
# # rewriter_model, rewriter_tokenizer = main_model, main_tokenizer

# # Load GSM8K training split and pick a subset
# raw_ds    = load_dataset("openai/gsm8k", "main", split="train").shuffle(seed=42)
# subset_sz = 50  # adjust as needed
# subset    = raw_ds.select(range(100,min(100+subset_sz, len(raw_ds))))

# # Build ORPO preference pairs
# records = []
# for sample in tqdm(subset):
#     orig_q  = sample["question"]
#     gt_ans  = extract_gt_answer(sample["answer"])
#     if gt_ans is None:
#         continue
#     # rewritten prompt
#     rewritten = create_prompts(orig_q, rewriter_model, rewriter_tokenizer)
#     # compute rewards for original vs rewritten
#     rew_reward  = compute_reward(orig_q, rewritten, main_model, main_tokenizer, gt_ans)
#     orig_reward = compute_reward(orig_q, orig_q,    main_model, main_tokenizer, gt_ans)
#     # determine chosen/rejected
#     if rew_reward >= orig_reward:
#         chosen, rejected = rewritten, orig_q
#     else:
#         chosen, rejected = orig_q, rewritten
#     records.append({"prompt": orig_q, "chosen": chosen, "rejected": rejected})


# import pandas as pd

# # Option B: Save as CSV
# df = pd.DataFrame(records)
# df.to_csv("orpo_dataset.csv", index=False)


In [45]:
# import pandas as pd

# loaded_df = pd.read_csv("orpo_dataset.csv")
# records = loaded_df.to_dict(orient="records")  # convert back to list of dicts if needed

In [46]:
# CONFIG.model_name = "unsloth/Llama-3.2-3B-Instruct"

# main_model, main_tokenizer = load_model_and_tokenizer(CONFIG.model_name, CONFIG.max_seq_length, inference=True)
# rewriter_model, rewriter_tokenizer = load_model_and_tokenizer(CONFIG.model_name, CONFIG.max_seq_length)
# # rewriter_model, rewriter_tokenizer = load_model_and_tokenizer(CONFIG.model_name, CONFIG.max_seq_length)

In [47]:
# rewriter_model = FastLanguageModel.get_peft_model(
#     rewriter_model,
#     r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 64,
#     lora_dropout = 0, # Currently only supports dropout = 0
#     bias = "none",    # Currently only supports bias = "none"
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

# # One must patch the DPO Trainer first!
# from unsloth import PatchDPOTrainer
# PatchDPOTrainer()